# Извлекаем изображения, таблицы и подписи к ним.

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %reset

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %run common_info.ipynb

### Этот код конвертирует страницы pdf-документа, содержащие таблицы, в jpeg-изображения.

In [ ]:
from pdf2image import convert_from_path

pages = []

for page in pagesWithTablesNumbers: # range(0, maxPages, 1):
    pages += convert_from_path(graduationWorkPDFFile, dpi=900, first_page=page, last_page=page)

for page in range(0, len(pagesWithTablesNumbers)): # range(0, maxPages):
    pages[page].save(temporaryFilesFolder + "page_"+str(pagesWithTablesNumbers[page])+".jpg", "JPEG")

### Этот код вытаскивает таблицы как jpeg-изображения.

In [ ]:
import cv2
from numpy import ndarray
import matplotlib.pyplot as plt

def extractTablesFromPage(graduationWorkJPGPagesFolder, graduationWorkTablesFolder, jpgPageFile, tableNumber):
    pageTablesY1 = []
    pageTablesY0 = []
    tablePaths = []

    # load source image
    srcImage = cv2.imread(graduationWorkJPGPagesFolder + jpgPageFile, cv2.IMREAD_GRAYSCALE)

    # resizing for practical reasons
    # resizedImage = cv2.resize(srcImage, (800, 900));
    # print(resizedImage.shape)

    # transform source image to gray if it is not
    # grayImage = cv2.cvtColor(resizedImage, cv2.COLOR_BGR2GRAY);

    # apply adaptiveThreshold at the bitwise_not of gray
    adaptiveThresholdToBitwiseNotGray = cv2.adaptiveThreshold(~srcImage, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -2);

    # create the images that will use to extract the horizontal and vertical lines
    horizontal = adaptiveThresholdToBitwiseNotGray.copy()
    vertical = adaptiveThresholdToBitwiseNotGray.copy()

    # play with this variable in order to increase/decrease the amount of lines to be detecte
    scale = 15


    # specify size on horizontal axis
    horizontalSize = horizontal.shape[1] / scale;

    # create structure element for extracting horizontal lines through morphology operations
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (int(horizontalSize), 1))
    # plt.imshow(horizontalStructure, cmap='gray')
    # plt.show()

    # apply morphology operations
    cv2.erode(horizontal, horizontalStructure, horizontal, (-1, -1))
    # expand horizontal lines
    cv2.dilate(horizontal, horizontalStructure, horizontal, (-1, -1))
    # plt.imshow(horizontal, cmap='gray')
    # plt.show()


    # specify size on vertical axis
    verticalSize = vertical.shape[0] / scale

    # create structure element for extracting vertical lines through morphology operations
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(verticalSize)))
    # plt.imshow(verticalStructure, cmap='gray')
    # plt.show()          

    # apply morphology operations
    cv2.erode(vertical, verticalStructure, vertical, (-1, -1))
    # expand vertical lines
    cv2.dilate(vertical, verticalStructure, vertical, (-1, -1))
    # plt.imshow(vertical, cmap='gray')
    # plt.show()


    # create a mask which includes the tables
    mask = horizontal + vertical;
    # plt.imshow(mask, cmap='gray')
    # plt.show()


    # find the joints between the lines of the tables, we will use this information in order to descriminate tables from pictures (tables will contain more than 4 joints while a picture only 4 (i.e. at the corners))
    joints = cv2.bitwise_and(horizontal, vertical);
    # plt.imshow(joints, cmap='gray')
    # plt.show()

    # cv2.imwrite("/home/arisu/Документы/Диплом/experiments/АграновскийМЛ_421/page_0.test.jpg", mask) 

    # find external contours from the mask, which most probably will belong to tables or to images
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contoursPoly = [None] * len(contours)
    boundRect = [None] * len(contours)
    rois = []

    for i in range(0, len(contours)):
        # find the area of each contour
        area = cv2.contourArea(contours[i])

        # filter individual lines of blobs that might exist and they do not represent a table
        # value is randomly chosen, you will need to find that by yourself with trial and error procedure
        if area < 100:
            continue

        contoursPoly[i] = cv2.approxPolyDP(contours[i], 3, True)
        boundRect[i] = cv2.boundingRect(contoursPoly[i])

        # find the number of joints that each table has
        r = boundRect[i]
        roi = joints[r[1]:r[1]+r[3], r[0]:r[0]+r[2]]
        jointsContours, jointsHierarchy = cv2.findContours(roi, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

        # if the number is not more than 5 then most likely it not a table
        if len(jointsContours) <= 4:
            continue

        r = boundRect[i]
        rois.append(srcImage[r[1]:r[1]+r[3], r[0]:r[0]+r[2]].copy())
        pageTablesY1.append((r[1]+r[3]) / srcImage.shape[0])
        pageTablesY0.append((r[1]) / srcImage.shape[0])

        # drawContours(rsz, contours, i, Scalar(0, 0, 255), CV_FILLED, 8, vector<Vec4i>(), 0, Point());
        cv2.rectangle(srcImage, (r[0], r[1]), (r[0]+r[2], r[1]+r[3]), (0, 255, 0), 1, 8, 0)

    for i in range(0, len(rois)):
        # now you can do whatever post process you want with the data within the rectangles/tables
        tableNumber += 1
        tablePath = graduationWorkTablesFolder + "table_" + str(tableNumber) + ".jpg"
        tablePaths.append({
            'path': tablePath,
            'height': rois[i].shape[0],
            'width': rois[i].shape[1]
        })
        cv2.imwrite(tablePath, rois[i])
        
    return pageTablesY1, pageTablesY0, tablePaths

In [ ]:
pageTablesY1List = []
pageTablesY0List = []
tablePaths = []

for page in range(0, maxPages):
    pageTablesY1List.append([])
    pageTablesY0List.append([])

for page in pagesWithTablesNumbers:
    pageTablesY1List[page-1], pageTablesY0List[page-1], pageTablePaths = extractTablesFromPage(temporaryFilesFolder, graduationWorkTablesFolder, "page_"+str(page)+".jpg", len(tablePaths))
    tablePaths += pageTablePaths

### Извлекаем подписи к таблицам (в строковом формате и в формате jpeg-изображения).

In [ ]:
from pdfminer.layout import LAParams, LTFigure, LTImage, LTTextBox, LTLine
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from PyPDF2 import PdfFileReader, PdfFileWriter
from copy import copy, deepcopy
from collections import OrderedDict
import re
from pyrutok import Token, Sentence, Tokenizer, GraphemTag, TokenType
from pymorphy2 import MorphAnalyzer

def extractTableCaptions(graduationWorkPDFFile, pageTablesY1List, pageTablesY0List, tablePaths):
    pageObjectsByY1List = [{}]
    captionsToTables = []
    morphAnalyzer = MorphAnalyzer()
    
    with open(graduationWorkPDFFile, 'rb') as graduationWork:
        resourceManager = PDFResourceManager()
        laParams = LAParams()
        pageAggregator = PDFPageAggregator(resourceManager, laparams=laParams)
        pageInterpreter = PDFPageInterpreter(resourceManager, pageAggregator)
        pages = list(PDFPage.get_pages(graduationWork))

        pdf = PdfFileReader(graduationWork)
        outputTableCaptions = PdfFileWriter()

        pageTablesY1ListCopy = deepcopy(pageTablesY1List)
        pageTablesY0ListCopy = deepcopy(pageTablesY0List)
        
        tablePathIndex = 0

        for page in range(1, len(pages)):       
            # нужна, чтобы сформировать новый pdf-файл (подписями к таблицам)
            pyPdfPage = pdf.getPage(page)
            # нужна, чтобы парсить pdf-файл на объекты
            pdfMinerPage = pages[page]
            pageInterpreter.process_page(pdfMinerPage)
            layout = pageAggregator.get_result()

            # переводим нижнюю границу таблиц в значения, соответствующие измерениям, генерируемым в pdfminer
            pageTablesY1 = pageTablesY1ListCopy[page]
            pageTablesY0 = pageTablesY0ListCopy[page]
            for tableY1Index in range(0, len(pageTablesY1)):
                pageHeight = pdf.getPage(page).mediaBox.getHeight()
                pageTablesY1[tableY1Index] = (1 - pageTablesY1[tableY1Index]) * float(pageHeight)
                pageTablesY0[tableY1Index] = (1 - pageTablesY0[tableY1Index]) * float(pageHeight)

            # объекты по их расположению на страницах по оси y
            pageObjectsByY1 = {}
            for ltObject in layout:
                x0, y0, x1, y1 = ltObject.bbox

                if y1 not in pageObjectsByY1.keys():
                    pageObjectsByY1[y1] = [{'label': '', 'object': ltObject}]
                else:
                    pageObjectsByY1[y1] += [{'label': '', 'object': ltObject}]

            pageObjectsByY1 = OrderedDict(sorted(pageObjectsByY1.items(), reverse=True))

            for tableY1, tableY0 in zip(pageTablesY1, pageTablesY0):
                # а теперь ищем подписи к таблицам
                for ltObjectY1 in pageObjectsByY1:
                    if tableY1 < ltObjectY1 < tableY0:
                        for ltObject in pageObjectsByY1[ltObjectY1]:
                            if isinstance(ltObject['object'], LTTextBox):
                                # отмечаем, что это не часть общего текста
                                    ltObject['label'] = 'table part'
                    if tableY1 > ltObjectY1:
                        for ltObject in pageObjectsByY1[ltObjectY1]:
                            if isinstance(ltObject['object'], LTTextBox):
                                text = ltObject['object'].get_text().strip() 
                                # 6 символов - это "Рис." и номер
                                if text != '' and len(text) > 6 and \
                                   len(set(text.lower()).intersection(letters)) > 0:
                                    # сохраняем как текст
                                    text = text.replace('-\n', '').replace('\n', ' ').replace('  ', ' ')
                                    # замеченные неточности в парсинге символов
                                    text = text.replace('(cid:22)', '-').replace('(cid:190)', '«').replace('¿', '»')
                                    
                                    # здесь записываем нумерацию таблицы, путь до изображения с ней и подпись к ней
                                    captionWordsAndNumbers = []
                                    for sentence in Tokenizer(text):
                                        for token in sentence:
                                            if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                                                word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                                                captionWordsAndNumbers.append(word)
                                            if token.get_token_type() == TokenType.NUMBER:
                                                captionWordsAndNumbers.append(token.get_escaped_data())
                                    captionBigrams = [captionWordsAndNumbers[i] + ' ' + captionWordsAndNumbers[i+1] 
                                                      for i in range(0, len(captionWordsAndNumbers)-1)]
                                    for bigram in captionBigrams:
                                        tableReference = re.findall(r'^(' + tableReferencesPattern + r')\ \d+$', bigram)
                                        imageReference = re.findall(r'^(' + imageReferencesPattern + r')\ \d+$', bigram)
                                        if tableReference:
                                            captionsToTables.append({
                                                'numeration': bigram.split(' ')[1],
                                                'filePath': tablePaths[tablePathIndex]['path'],
                                                'height': tablePaths[tablePathIndex]['height'],
                                                'width': tablePaths[tablePathIndex]['width'],
                                                'caption': text
                                            })
                                            break
                                        elif imageReference:
                                            captionsToTables.append({
                                                'numeration': bigram.split(' ')[1],
                                                'filePath': tablePaths[tablePathIndex]['path'],
                                                'height': tablePaths[tablePathIndex]['height'],
                                                'width': tablePaths[tablePathIndex]['width'],
                                                'caption': text
                                            })
                                            break
                                    else:
                                        captionsToTables.append({
                                            'numeration': '?',
                                            'filePath': tablePaths[tablePathIndex]['path'],
                                            'height': tablePaths[tablePathIndex]['height'],
                                            'width': tablePaths[tablePathIndex]['width'],
                                            'caption': text
                                        })
                                    tablePathIndex += 1

                                    # сохраняем как картинку
                                    pyPdfPageCopy = copy(pyPdfPage)
                                    x0, y0, x1, y1 = ltObject['object'].bbox
                                    pyPdfPageCopy.mediaBox.lowerLeft = (x0, y0)
                                    pyPdfPageCopy.mediaBox.upperRight = (x1, y1)
                                    outputTableCaptions.addPage(pyPdfPageCopy)

                                    # отмечаем, что это не часть общего текста
                                    ltObject['label'] = 'table caption'
                                    break
                        else:
                            continue
                        break

            pageObjectsByY1List.append(pageObjectsByY1)

        with open(temporaryFilesFolder + "table_captions.pdf",'wb') as captionsPDF:
            outputTableCaptions.write(captionsPDF)
        
    return pageObjectsByY1List, captionsToTables

In [ ]:
from pprint import pprint

pageObjectsByY1List, captionsToTables = extractTableCaptions(graduationWorkPDFFile, pageTablesY1List, pageTablesY0List, tablePaths)

if __name__ == '__main__' and '__file__' not in globals():
    pprint(captionsToTables)

### Извлекаем изображения и подписи к ним (в строковом формате и в формате png-изображения).

<ins>Примечание</ins>: исключаем те изображения, которые похожи таблицы. Это позволяет извлекать таблицы отдельно, если они не сверстаны в редакторе, а вставлены как картинки. Однако если изображение похоже на таблицу, но не является ей (например, ограничено черной рамкой), то оно будет извлечено как таблица. Это создает сложности, если такое избражение представляет из себя группу изображений, ограниченных черной рамкой и вместе похожих на таблицу.

In [ ]:
from pdfminer.layout import LAParams, LTFigure, LTImage, LTTextBox, LTLine
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from PyPDF2 import PdfFileReader, PdfFileWriter
from copy import copy
from collections import OrderedDict

def extractImagesAndCaptions(graduationWorkPDFFile, pageObjectsByY1List):
    captionsToImages = []
    
    with open(graduationWorkPDFFile, 'rb') as graduationWork:
        resourceManager = PDFResourceManager()
        laParams = LAParams()
        pageAggregator = PDFPageAggregator(resourceManager, laparams=laParams)
        pageInterpreter = PDFPageInterpreter(resourceManager, pageAggregator)
        pages = list(PDFPage.get_pages(graduationWork))

        pdf = PdfFileReader(graduationWork)
    
        outputImages = PdfFileWriter()
        outputCaptions = PdfFileWriter()

        for page in range(1, len(pages)):
            # нужна, чтобы сформировать новый pdf-файл (с картинками или подписями к ним)
            pyPdfPage = pdf.getPage(page)
            # нужна, чтобы парсить pdf-файл на объекты
            pdfMinerPage = pages[page]
            pageInterpreter.process_page(pdfMinerPage)
            layout = pageAggregator.get_result()

            # объекты по их расположению на страницах по оси y
            pageObjectsByY1 = pageObjectsByY1List[page]

            for ltObject in layout:
                # если это изображение, то сохраняем его в картинку (таблицы обрабытываются отдельно)
                if isinstance(ltObject, LTFigure) or \
                   isinstance(ltObject, LTImage):
                    pyPdfPageCopy = copy(pyPdfPage)
                    x0, y0, x1, y1 = ltObject.bbox

                    pyPdfPageCopy.mediaBox.lowerLeft = (x0, y0)
                    pyPdfPageCopy.mediaBox.upperRight = (x1, y1)

                    outputImages.addPage(pyPdfPageCopy)

                    # отмечаем, что это изображение
                    for ltObj in pageObjectsByY1[y1]:
                        if ltObj['object'] == ltObject:    
                            ltObj['label'] = 'image'

                    # находим расположение по оси y очередного обработанного изображения
                    imageIndexInSortedObjectsY1List = list(pageObjectsByY1.keys()).index(y1)
    #                 print(x1, objectsX1List) 
                    # а теперь ищем (с зафиксированного места) подпись к этому изображению
                    for index in range(imageIndexInSortedObjectsY1List, len(pageObjectsByY1)):
                        for ltObj in pageObjectsByY1[list(pageObjectsByY1.keys())[index]]:
                            if isinstance(ltObj['object'], LTTextBox):
                                text = ltObj['object'].get_text().strip() 
                                # 6 символов - это "Рис." и номер
                                if text != '' and len(text) > 6 and \
                                   len(set(text.lower()).intersection(letters)) > 0:
                                    # рассматриваемое изображение - это таблица, которую мы уже извлекали другим способом
                                    # тогда соответствующее изображение и подпись к нему не нужны
                                    if (ltObj['label'] == 'table caption'):
                                        # удаляем соответствующее изображение
                                        outputCaptionsTemp = PdfFileWriter()
                                        for page in range(0, outputCaptions.getNumPages()-1):
                                            outputCaptionsTemp.addPage(outputCaptions.getPage(page))
                                        outputCaptions = outputCaptionsTemp
                                        break

                                    # сохраняем как текст
                                    text = text.replace('-\n', '').replace('\n', ' ').replace('  ', ' ')
                                    # замеченные неточности в парсинге символов
                                    text = text.replace('(cid:22)', '-').replace('(cid:190)', '«').replace('¿', '»')
                                    captionsToImages += [text]

                                    # сохраняем как картинку
                                    pyPdfPageCopy = copy(pyPdfPage)
                                    x0, y0, x1, y1 = ltObj['object'].bbox
                                    pyPdfPageCopy.mediaBox.lowerLeft = (x0, y0)
                                    pyPdfPageCopy.mediaBox.upperRight = (x1, y1)
                                    outputCaptions.addPage(pyPdfPageCopy)

                                    # отмечаем, что это не часть общего текста
                                    ltObj['label'] = 'image caption'
                                    break
                        else:
                            continue
                        break

        with open(temporaryFilesFolder + "images.pdf",'wb') as imagesPDF:
            outputImages.write(imagesPDF)
        with open(temporaryFilesFolder + "image_captions.pdf",'wb') as captionsPDF:
            outputCaptions.write(captionsPDF)
    
    return pageObjectsByY1List, captionsToImages

In [ ]:
from pprint import pprint

pageObjectsByY1List, captionsToImagesTemp = extractImagesAndCaptions(graduationWorkPDFFile, pageObjectsByY1List)

if __name__ == '__main__' and '__file__' not in globals():
    pprint(captionsToImagesTemp)

### Сохраняем изображения в отдельную папку.

In [ ]:
from pdf2image import convert_from_path
from itertools import count
from PIL import Image
from pyrutok import Token, Sentence, Tokenizer, GraphemTag
from pymorphy2 import MorphAnalyzer

pages = []
intNumbersGenerator = count(1)
morphAnalyzer = MorphAnalyzer()
captionsToImages = []

for page in range(0, len(captionsToImagesTemp)): 
    pages += convert_from_path(temporaryFilesFolder + "images.pdf", dpi=500, first_page=page, last_page=page)

for page in range(0, len(captionsToImagesTemp)):
    imagePath = graduationWorkImagesFolder + "image_"+str(next(intNumbersGenerator))+".png"
    pages[page].save(imagePath, "PNG")
    image = Image.open(imagePath)
    
    # здесь записываем нумерацию изображения, путь до файла с ним и подпись к нему
    captionWordsAndNumbers = []
    for sentence in Tokenizer(captionsToImagesTemp[page]):
        for token in sentence:
            if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                captionWordsAndNumbers.append(word)
            if token.get_token_type() == TokenType.NUMBER:
                captionWordsAndNumbers.append(token.get_escaped_data())
    captionBigrams = [captionWordsAndNumbers[i] + ' ' + captionWordsAndNumbers[i+1] 
                      for i in range(0, len(captionWordsAndNumbers)-1)]
    captionTrigrams = [captionWordsAndNumbers[i] + ' ' + captionWordsAndNumbers[i+1] + ' ' + captionWordsAndNumbers[i+2]
                      for i in range(0, len(captionWordsAndNumbers)-2)]
    for trigram in captionTrigrams:
        imageReference = re.findall(r'^(' + imageReferencesPattern + r')\ \d+\ \d+$', trigram)
        if imageReference:
            captionsToImages.append({
                'numeration': trigram.split(' ', 1)[1],
                'filePath': imagePath,
                'height': image.size[1],
                'width': image.size[0],
                'caption': captionsToImagesTemp[page]
            })
            break
    else:
        for bigram in captionBigrams:
            imageReference = re.findall(r'^(' + imageReferencesPattern + r')\ \d+$', bigram)
            if imageReference:
                captionsToImages.append({
                    'numeration': bigram.split(' ')[1],
                    'filePath': imagePath,
                    'height': image.size[1],
                    'width': image.size[0],
                    'caption': captionsToImagesTemp[page]
                })
                break
        else:
            captionsToImages.append({
                'numeration': '?',
                'filePath': imagePath,
                'height': image.size[1],
                'width': image.size[0],
                'caption': captionsToImagesTemp[page]
            })

if __name__ == '__main__' and '__file__' not in globals():
    pprint(captionsToImages)
        
if not (__name__ == '__main__' and '__file__' not in globals()):
    captionsToImagesTemp = None
    pages = None
    intNumbersGenerator = None
    morphAnalyzer = None